In [46]:
import sys
print(sys.executable)
!conda info | grep 'active env'

/sc/arion/work/massen06/.conda/envs/qc_torch/bin/python3.9
     active environment : qc_torch
    active env location : /sc/arion/work/massen06/.conda/envs/qc_torch


In [47]:
import os
import copy
import scipy
import shutil
import logging
import h5py
import tables
import anndata as ad
import scanpy as sc
import pandas as pd
import numpy as np
from datetime import datetime
import scipy.sparse as sparse


# Mouse Living Brain - Data Consolidation

Nicolas Masse, Donghoon Lee  
July 2023  

Assumes ambient mRNA has already been removed using ambient_removal.ipynb. This step will save the data in .h5 format in the base_dir (see below).

### Paths for metadata spreadsheet, gene info, and target path for AnnData

In [48]:
metadata_dir = "/sc/arion/projects/CommonMind/single-cell-neurogenomics/LIVING_BRAIN/metadata"
base_dir = "/sc/arion/work/massen06/living_brain1/ambient_removal"
save_dir = "/sc/arion/work/massen06/living_brain1/anndata"
gene_info_fn = "./data/MM_GRCm39_full.txt" 
mouse_mitocarta_fn = "./data/MouseMitoCarta3.0.csv" 

### Extract metadata from Living_brain_snRNA_template.xlsx


In [49]:
metadata_fn = os.path.join(metadata_dir, "Living_brain_snRNA_template.xlsx")
metadata = pd.read_excel(metadata_fn, sheet_name="Mouse_cDNA_Libraries")
#metadata = pd.read_excel(metadata_fn)
xl = pd.ExcelFile(metadata_fn)
xl.sheet_names  # see all sheet names

['Mouse sample Info',
 'Human sample Info',
 'Master Day 1',
 'Evelyn',
 'Clara',
 'Mouse_cDNA_Libraries',
 'Human_cDNA_libraries',
 'HTOs',
 'Pooling_rand',
 'Pooling',
 'Barcodes',
 'Ab Barcodes']

In [50]:
def extract_donor_information(data):
    """Extract donor name, PMI and living status"""
    
    donor_info = {
        "unique_id": [],
        "SubID" : [],
        "pmi": [],
        "living": [],
    }

    for d in data.values:
        
        # get donor name
        i0 = d.find("_")
        donor_info["SubID"].append(d[: i0])
        
        # get PMI
        time = int("".join([s for s in d[i0 :] if s.isdigit()]))
        if "hr" in d[i0 :]:
            time *= 60
        donor_info["pmi"].append(time)
        
        if "PM" in d or "Post" in d or time > 0:
            donor_info["living"].append("PM")
        else:
            donor_info["living"].append("Living")
            
        un_id = donor_info["SubID"][-1] + "_" +  donor_info["living"][-1]
        donor_info["unique_id"].append(un_id)
        
    return donor_info

In [51]:
for k in metadata.keys():
    
    column_name = metadata[k].iloc[0]

    if column_name == "unique sample ID":
        donor_info = extract_donor_information(metadata[k].iloc[1 :])
        
    elif column_name == "Step 2 date":
        dates = metadata[k].iloc[1 :]
        dates = [d.strftime("%m%d%Y") for d in dates]
        donor_info["step_2_date"] = dates
        
    elif column_name == "cDNA Amp cycle #s":
        cycles = metadata[k].iloc[1 :]
        cycles = [int(c) for c in cycles]
        donor_info["cdna_amp_cycle"] = cycles
        
    elif column_name == "input (ng)":
        inputs = metadata[k].iloc[1 :]
        inputs = [round(float(i), 1) for i in inputs]
        donor_info["input_ng"] = inputs
        
    elif column_name == "Indexing PCR cycle #":
        cycle = metadata[k].iloc[1 :]
        cycle = [int(c) for c in cycle]
        donor_info["pcr_cycle"] = cycle
        
donor_info = pd.DataFrame(data = donor_info)

print(donor_info)  

     unique_id SubID  pmi  living step_2_date  cdna_amp_cycle  input_ng  \
0   WTA_Living   WTA    0  Living    06022023              11     139.6   
1       WTA_PM   WTA    0      PM    06022023              11     174.0   
2   912_Living   912    0  Living    06022023              11      40.8   
3       912_PM   912    5      PM    06022023              11     146.8   
4   WTF_Living   WTF    0  Living    06022023              11      31.4   
5       WTF_PM   WTF   60      PM    06022023              11     140.4   
6   WTB_Living   WTB    0  Living    06062023              12      28.0   
7       WTB_PM   WTB    0      PM    06062023              12      40.0   
8   913_Living   913    0  Living    06062023              12      21.4   
9       913_PM   913    5      PM    06062023              12      47.6   
10  WTE_Living   WTE    0  Living    06062023              12      16.0   
11      WTE_PM   WTE  360      PM    06062023              12      50.0   
12  WTC_Living   WTC    0

### Load gene info from Biomart

In [52]:
gene_info = pd.read_csv(gene_info_fn, sep='\t')
gene_info.drop_duplicates(subset = "Gene name", inplace=True)
gene_info.reset_index(inplace=True)
gene_info

/tmp/ipykernel_75251/2442110492.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  gene_info = pd.read_csv(gene_info_fn, sep='\t')


,index,Gene stable ID,Gene stable ID version,Transcript stable ID,Transcript stable ID version,Gene start (bp),Gene end (bp),Gene name,Gene type,Gene description,Chromosome/scaffold name
0,0,ENSMUSG00000064336,ENSMUSG00000064336.1,ENSMUST00000082387,ENSMUST00000082387.1,1,68,mt-Tf,Mt_tRNA,mitochondrially encoded tRNA phenylalanine [So...,MT
1,1,ENSMUSG00000064337,ENSMUSG00000064337.1,ENSMUST00000082388,ENSMUST00000082388.1,70,1024,mt-Rnr1,Mt_rRNA,mitochondrially encoded 12S rRNA [Source:MGI S...,MT
2,2,ENSMUSG00000064338,ENSMUSG00000064338.1,ENSMUST00000082389,ENSMUST00000082389.1,1025,1093,mt-Tv,Mt_tRNA,mitochondrially encoded tRNA valine [Source:MG...,MT
3,3,ENSMUSG00000064339,ENSMUSG00000064339.1,ENSMUST00000082390,ENSMUST00000082390.1,1094,2675,mt-Rnr2,Mt_rRNA,mitochondrially encoded 16S rRNA [Source:MGI S...,MT
4,4,ENSMUSG00000064340,ENSMUSG00000064340.1,ENSMUST00000082391,ENSMUST00000082391.1,2676,2750,mt-Tl1,Mt_tRNA,mitochondrially encoded tRNA leucine 1 [Source...,MT
...,...,...,...,...,...,...,...,...,...,...,...
56477,149518,ENSMUSG00000054766,ENSMUSG00000054766.14,ENSMUST00000134364,ENSMUST00000134364.8,29947390,29962589,Set,protein_coding,SET nuclear oncogene [Source:MGI Symbol;Acc:MG...,2
56478,149525,ENSMUSG00000026785,ENSMUSG00000026785.15,ENSMUST00000125346,ENSMUST00000125346.8,29967696,29981034,Pkn3,protein_coding,protein kinase N3 [Source:MGI Symbol;Acc:MGI:2...,2
56479,149533,ENSMUSG00000015335,ENSMUSG00000015335.17,ENSMUST00000148717,ENSMUST00000148717.8,29980956,29983660,Zdhhc12,protein_coding,"zinc finger, DHHC domain containing 12 [Source...",2
56480,149537,ENSMUSG00000039686,ENSMUSG00000039686.15,ENSMUST00000044751,ENSMUST00000044751.14,29987295,30014597,Zer1,protein_coding,"zyg-11 related, cell cycle regulator [Source:M...",2


In [53]:
def link_gene_info(adata, gene_info, verbose=False):
    """Add gene info to AnnData"""
    
    adata.var = adata.var.merge(gene_info, left_on="gene_ids", right_on='Gene stable ID', how="outer")

    good_genes_idx = [i for i, name in enumerate(adata.var["Gene name"].values) if isinstance(name, str)]
       
    new_adata = ad.AnnData(adata.X[:, good_genes_idx])
    new_adata.obs = adata.obs
    new_adata.var["gene_name"] = [v for n, v in enumerate(adata.var["Gene name"].values) if n in good_genes_idx]
    new_adata.var["gene_type"] = [v for n, v in enumerate(adata.var["Gene type"].values)  if n in good_genes_idx]
    new_adata.var["gene_id"] = [v for n, v in enumerate(adata.var["Gene stable ID"].values)  if n in good_genes_idx]
    new_adata.var["gene_chrom"] = [v for n, v in enumerate(adata.var["Chromosome/scaffold name"].values)  if n in good_genes_idx]
    new_adata.var["gene_start"] = [v for n, v in enumerate(adata.var["Gene start (bp)"].values)  if n in good_genes_idx]
    new_adata.var["gene_end"] = [v for n, v in enumerate(adata.var["Gene end (bp)"].values)  if n in good_genes_idx]
    
    print(f"Variables added. Number of genes remaining: {new_adata.X.shape[1]}")
    
    return new_adata

In [54]:
def readCellbenderH5(filename):
    
    f = h5py.File(filename, 'r')
    mat = f['matrix']
    cols = ['barcodes','latent_cell_probability','latent_RT_efficiency']
    #cols = ["barcodes"]
    obsdict = {x:mat[x] for x in cols}
    
    adata=sc.AnnData(X=scipy.sparse.csr_matrix(
        (
            mat['data'][:], 
            mat['indices'][:], 
            mat['indptr'][:]
        ), 
        shape=(mat['shape'][1],mat['shape'][0])),
        var=pd.DataFrame(dict(mat['features'])),
        obs=pd.DataFrame(obsdict),
        #uns={'test_elbo':list(mat['test_elbo']),'test_epoch':list(mat['test_epoch'])}
    )
    for k in adata.var.keys():
        adata.var[k]=[x.decode('ascii') for x in adata.var[k]]
        
    adata.var.index = [x for x in adata.var['name']]
    adata.obs.barcodes = [str(x.decode('ascii')) for x in mat['barcodes']]
    adata.obs.index = adata.obs.barcodes
    #adata.obs.index = [x.decode('ascii') for x in mat['barcodes']]
    #adata.obs.reset_index(adata.obs.barcodes)
    
    return adata

In [55]:
def readCellbenderH5(filename, suffix):
    
    f = h5py.File(filename, 'r')
    mat = f['matrix']
    
    x = scipy.sparse.csr_matrix(
        (
            mat['data'][:], 
            mat['indices'][:], 
            mat['indptr'][:]
        ), 
        shape=(mat['shape'][1],mat['shape'][0])
    )
    gene_dict = dict(mat['features'])
    
    barcodes = np.array([x.decode('ascii') + f"-{str(suffix)}" for x in mat['barcodes']])
    _, idx = np.unique(barcodes, return_index=True)
    x = x[idx, :]
    barcodes = barcodes[idx]
    
    return x, barcodes, gene_dict
    

### Create AnnData from filtered samples with ambient mRNA removed

In [56]:
filtered_fns = [fn for fn in os.listdir(base_dir) if "_filtered.h5" in fn]
print(f"Number of filtered data files found: {len(filtered_fns)}")

Number of filtered data files found: 18


In [57]:
x = None
cell_counts = []

for n, fn in enumerate(filtered_fns):

    # Load filtered data (ambient mRNA already removed)
    x_temp, barcodes_temp, gene_dict = readCellbenderH5(os.path.join(base_dir, fn), n)
    n_cells = len(barcodes_temp)
    cell_counts.append(x_temp.shape[0])
    
    
    # Find the matching donor ID
    idx = fn.find("filtered")
    sample_id = fn[: idx - 1] 
    
    i0 = fn.find("_")
    donor = fn[: i0]
    pmi = int("".join([s for s in fn[i0 :-2] if s.isdigit()]))
    living = "PM" if "PM" in fn or "Post" in fn or pmi > 0 else "Living"
    
    unique_id = donor + "_" + living 
    
    donor_idx = [i for i, donor_id in enumerate(donor_info.unique_id) if unique_id == donor_id]
    assert len(donor_idx) <= 1, "Multiple matches for sample ID"
    print(fn, unique_id, donor_idx, living, pmi)

        
    if x is None:
        x = copy.deepcopy(x_temp)
        obs = {"barcodes": list(barcodes_temp)}
        for k, v in donor_info.iloc[donor_idx[0]].items():
            obs[k] = [v] * n_cells

    else:        
        x = sparse.vstack([x, x_temp])
        for k, v in donor_info.iloc[donor_idx[0]].items():
            obs[k] += [v] * n_cells
        obs["barcodes"] +=  list(barcodes_temp)

        
adata = sc.AnnData(x)
for k, v in obs.items():
    adata.obs[k] = v
    
adata.var = pd.DataFrame({"gene_ids": [g.decode('ascii') for g in gene_dict["id"]]})
#adata.var.gene_ids = [g.decode('ascii') for g in gene_dict["id"]]
    

adata = link_gene_info(adata, gene_info)
adata.var.index = adata.var.gene_name

adata.var.gene_chrom = adata.var.gene_chrom.astype('str')
adata.var_names_make_unique()

anndata_fn = os.path.join(save_dir, "mouse_living_brain_filtered.h5ad")
adata.write(anndata_fn)
print(f"Saved {anndata_fn}")

WTC_0min_PM_filtered.h5 WTC_PM [13] PM 0
WTG_1hr_filtered.h5 WTG_PM [15] PM 1
WTD_6hr_Post_filtered.h5 WTD_PM [17] PM 6
WTB_0min_PM_filtered.h5 WTB_PM [7] PM 0
WTF_1hr_filtered.h5 WTF_PM [5] PM 1
WTB_0min_Living_PFC_filtered.h5 WTB_Living [6] Living 0
913_0min_filtered.h5 913_Living [8] Living 0
WTG_0min_filtered.h5 WTG_Living [14] Living 0
WTE_6hr_Post_filtered.h5 WTE_PM [11] PM 6
WTA_0min_Living_PFC_filtered.h5 WTA_Living [0] Living 0
WTE_0min_Living_filtered.h5 WTE_Living [10] Living 0
WTD_0min_Living_filtered.h5 WTD_Living [16] Living 0
WTC_0min_Living_filtered.h5 WTC_Living [12] Living 0
WTA_0min_PM_filtered.h5 WTA_PM [1] PM 0
912_0min_filtered.h5 912_Living [2] Living 0
912_5min_filtered.h5 912_PM [3] PM 5
WTF_0min_filtered.h5 WTF_Living [4] Living 0
913_5min_filtered.h5 913_PM [9] PM 5
Variables added. Number of genes remaining: 56481
Saved /sc/arion/work/massen06/living_brain1/anndata/mouse_living_brain_filtered.h5ad


In [45]:
adata

AnnData object with n_obs × n_vars = 80703 × 56481
    obs: 'barcodes', 'unique_id', 'SubID', 'pmi', 'living', 'step_2_date', 'cdna_amp_cycle', 'input_ng', 'pcr_cycle'
    var: 'gene_name', 'gene_type', 'gene_id', 'gene_chrom', 'gene_start', 'gene_end'

In [60]:
print(np.min(cell_counts), np.max(cell_counts), np.median(cell_counts))

707 6525 4779.5
